In [5]:
#This notebook contains some scripts that are useful for cubechi2 and feria

In [6]:
import numpy as np

In [7]:
#Load up your lists from Feria
#For Keplerian Only sets, Rcb = max value in Rin
DList = ['300']
MList = ['1.6','1.7','1.8','1.9','2.0','2.1','2.2','2.3','2.4','2.5','2.6','2.7','2.8','2.9','3.0']
CBList = ['500.','450.','400.','350.','300.','250.','200.','150.','100.']
IList = ['89']
PAList = ['57.5']
RotList = ['1', '-1']
RoutList = ['1000.','900.','800.','700.','600.','500.','400.','300.','200.','100.'] 
RinList = ['1.', '100.', '200.', '300.'] 

In [8]:
def numberOfCubes(MassList, RotationList, CentBList, RouterList, RinnerList):
    #cubes=len(MassList) * len(RotationList)
    cubes = 0
    for i in range(len(RouterList)): #Scan through Router
        
            for j in range(len(RinnerList)): #Scan through Rinner
            
                if float(RouterList[i]) >= float(RinnerList[j]): #Make sure Rout>Rin
                
                    for k in range(len(CentBList)): #Scan through CB
                        
                        if ((float(CentBList[k]) >= float(RinnerList[j])) & (float(CentBList[k])<=float(RouterList[i]))):
                            
                            cubes+=1
                            
                        else:
                            pass
                else:
                    pass
    else:
          pass
                            
                        
                
                
    
    
    
    
    cubes = cubes * len(MassList) * len(RotationList)
    return cubes

In [9]:
def falseNumberOfCubes(MassList, RotationList, CentBList, RouterList, RinnerList):
    return(len(MassList)*len(RotationList)*len(CentBList)*len(RouterList)*len(RinnerList))

In [10]:
#This cell will print off all the information

print('The correct number of cubes is ',numberOfCubes(MList, RotList, CBList, RoutList, RinList))
print('Feria will say there is ',falseNumberOfCubes(MList, RotList, CBList, RoutList, RinList))
print('That is a reduction of ', (100*(1-(numberOfCubes(MList, RotList, CBList, RoutList, RinList)/falseNumberOfCubes(MList, RotList, CBList, RoutList, RinList)))), '%')
print('At roughly 8.4MB per cube, there will need to be storage for ', numberOfCubes(MList, RotList, CBList, RoutList, RinList)*8.4,'MB, or ', np.round(numberOfCubes(MList, RotList, CBList, RoutList, RinList)*8.4 * 10**(-6), 6), 'TB')
print('At roughly 10 seconds per cube, this will take ', numberOfCubes(MList, RotList, CBList, RoutList, RinList) * 10, 'seconds or ', numberOfCubes(MList, RotList, CBList, RoutList, RinList) * 10/60, 'minutes to complete')


The correct number of cubes is  6750
Feria will say there is  10800
That is a reduction of  37.5 %
At roughly 8.4MB per cube, there will need to be storage for  56700.0 MB, or  0.0567 TB
At roughly 10 seconds per cube, this will take  67500 seconds or  1125.0 minutes to complete


This is the handeling of the input files

In [13]:
#This script will generate a .txt file of the names of all image cubes

import glob #this package allows for the searching of files and outputs their filepaths
from numpy import char
ListoNames=glob.glob('/Volumes/REU_2023/Per-emb-8_Models/Per-emb-8_Odd_MList_REDO/IREOnly/*9.78.fits') #Grab filepaths for all image cubes
for i in range(len(ListoNames)):
    if ListoNames[i][0:14] == 'cubechi2/fits/':
        ListoNames[i] = ListoNames[i][14:]
    elif ListoNames[i][0:9] == 'cubechi2/':
        ListoNames[i] = ListoNames[i][9:]
ListoNames=np.array(ListoNames) #Send it to an np.array
for i in range(len(ListoNames)):
    ListoNames[i]='\n'+ListoNames[i] #Adding a newline character to separate the filepaths out
    if ListoNames[i][-1] != 's': #Fixing the missing 's' in 'fits' in some files
        temp = ''
        temp = ListoNames[i]+'s\n'
        ListoNames[i]=temp
        #print(ListoNames[i], temp)
file=open('ListofNames.txt', 'w') #Making a new .txt file to write to
for i in range(len(ListoNames)): #Writing all the filepaths with their newline characters to the file
    file.writelines(ListoNames[i])
file.close() #And done


#EDIT FILE TO REMOVE FIRST EMPTY LINE
#ALSO, FIND AND REPLACE ALL .fit WITH .fits BUT KEEP THE MASSSIVE NEWLINE i.e. COPY FROM START OF NEXT NAME TO THE . IN .fits
#Also remove filepath from all files, should just be the actual name of the fits file, no directory
#...20.25.fits%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#%PER-EMB... THE % SHOULD BE SELECTED SPACES

In [9]:
def TemplateGroup(ModelFilename, ModelFilepath, ObservedFilename, ObservedFilepath, Noise, Cutoff, ObservingArea=np.zeros(5), OffsetVelocity = 0, Weight=0):
    '''
    This function will write out a complete 'chunk' for the input file for cubechi2, using filenames already made
    This is to bypass the issues seen with cubechi2 making filenames already generated, and messing that up
    
    Inputs
    -----------------------
    ModelFilename: Single string, filename of the FERIA model cube
    ModelFilepath: Single string, take the filepath to the files, starting after /Volumes/. Should end with the last /
    ObservedFilename: Single string, observed data being compared to the model.
    ObservedFilepath: Single string, observed data filepath starting after /Volumes/. End on / like ModelFilename
    OffsetVelocity: Float m/s, How much to adjust the model before comparison for systemic velocity. Should for our case be 0 so defaults(edit: LEAVE IT 0 AT ALL TIMES)
    Noise: float Jy/beam, RMS noise level
    Cutoff: Threshold to cutoff data below noise level
    ObservingArea: 1D array 6 entries, RA, Dec, Velocity min and max values
    Weight: Weighting on chi2 calculation, will default to 0(which tells cubechi to figure it out), but setting it manually here is an option also
    
    Output
    -----------------------
    8 Line text, properly formatted such that concatination keeps the input file valid. Simply rename to .in, use in cubechi2
    '''
    
    ModelFilenameEdit = ModelFilename[:-6]     #removes .fits from filename
    line1 = ModelFilepath + ModelFilenameEdit + '.out\n' 
    line2 = ObservedFilepath + ObservedFilename + '\n'
    line3 = ModelFilepath + ModelFilename
    line4 = str(OffsetVelocity)+'    # Offset for Systemic Velocity (m/s)\n'
    line5 = str(Noise) + ' ' + str(Cutoff)+'   # rms (Jy/beam), threshold (Jy/beam)\n'
    line6 = str(ObservingArea[0]) + ' ' +  str(ObservingArea[1]) + ' '+ str(ObservingArea[2]) + ' ' + str(ObservingArea[3]) + ' ' + str(ObservingArea[4]) + ' ' + str(ObservingArea[5]) +  '    # Ramin, Ramax, Decmin, Decmax, Vmin, Vmax\n'
    line7 = str(Weight) + '    # Weight // If this value is <=0, weight is calculated\n' 
    line8 = '\n'
    CombinedParagraph = line1+line2+line3+line4+line5+line6+line7+line8
    return CombinedParagraph
    

In [14]:
#This script will let you read in the list of filenames, and compile that into a input file form
names=open('ListofNames.txt')
file=open('Per8MassCenterIREREDO.txt', 'w')
testsetup=names.readlines()
for i in range(len(testsetup)):
    paragraph = TemplateGroup(testsetup[i], #Model cube name
                              'REU_2023/Per-emb-8_Models/Per-emb-8_Odd_MList_REDO/IREOnly/', #filepath to model cube
                              'Per-emb-8_vel_subcube_centercut.fits', #name of the observed data
                              'REU_2023/Per-emb-8_Models/', #filepath of the observed data #NO /VOLUMES/
                              .005, #noise
                              4*.005, #threshold
                              [0,360,-180,180,7933.26,13224.9]) #coordinate stuffs
                                                #^these 2^ numbers should change, VMin and VMax in Data Logging Spreadsheet
    file.writelines(paragraph)
file.close()

#Somehow it makes the last entry have the systemic velocity move up a line, go to last paragraph and manually
#hit enter separating the lines. The first blank line after a paragraph should be on a multiple of 8

#Also, make sure for input modelcube to add 'fits/' in front of the filename in texteditor